**Analysis**

In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
# Google Colab Specific
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NLP_Ass_GC/project-group16/tuwnlpie/milestone2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP_Ass_GC/project-group16/tuwnlpie/milestone2


**Setting up the basic (not additionally trained) version of the model:**

In [53]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

def analyse_text(text):
  torch.manual_seed(456)
  torch.cuda.manual_seed_all(456)
  np.random.seed(456)
  tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
  model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  print('Predicted class is:', model.config.id2label[predicted_class_id])
  probabilities = torch.nn.functional.softmax(logits, dim=1)
  print(f'Probability for NEGATIVE  is {probabilities.tolist()[0][0]:.2f}, and probability for POSITIVE is {probabilities.tolist()[0][1]:.2f}')

Mismatches are stored in the csv, along with the text, true label and the confidence of annotation.

In [4]:
df_mismatches = pd.read_csv('../../data/mismatches.csv')
df_mismatches.head(5)

idx                                               text  confidence  real_Y
0    0  Its like that, if you want or not.“ ME: I have...         0.8       0
1    4  I haven’t said anything to him yet because I’m...         0.8       0
2   15  Despite being younger than him, I have had mor...         0.8       0
3   18  If I go to an interview for example, I'll know...         0.6       0
4   19  Like sleep would never be a simple thing for m...         0.0       0

When checking the distribution of misclassification per confidence values and true label values, it can be observed that more label with the value 0 gets misclassified more often. However, the reason behind this could also be the imbalance of the dataset.

In [23]:
grouped_df = df_mismatches.groupby(['real_Y', 'confidence']).count().reset_index()
grouped_df

real_Y  confidence  idx  text
0        0    0.000000   33    33
1        0    0.600000   39    39
2        0    0.666667    3     3
3        0    0.714286    1     1
4        0    0.800000   26    26
5        0    0.833333    5     5
6        0    1.000000   27    27
7        1    0.000000    2     2
8        1    0.500000    2     2
9        1    0.600000    7     7
10       1    0.666667    3     3
11       1    0.833333    1     1
12       1    1.000000    2     2

From the table shown above, we can see that there are a lot of misclassified observations with the confidence level 0. In total, as listed bellow, 35 of instances. Additionally, there could be some ambiguity to the values with the lower confidence values, in the range of 0.5 to 0.6.

In [24]:
filtered_df_conf_0 = df_mismatches[df_mismatches['confidence'] == 0.0]
print(f"Number of instances with confidence = 0.0: {filtered_df_conf_0.shape[0]}")

Number of instances with confidence = 0.0: 35


It is also interesting to look into specific examples with very high confidence values. Below, we can examine such instances along with their corresponding prediction probabilities.

Misclassification may be attributed to the presence of words with inherently negative connotations, such as anxiety, negativity, guilt, drugs, and others. This semantic complexity poses a challenge for the model to accurately classify the content.

Additionaly, the results come from the raw (not additionally trained) version of the model. Since Stress Detection is different than the task of labeling texts as positive or negative, it is natural that some discrepancies will prevail. In the end, these outputs also differ from the outputs that may be obtained from the real model.

Here we can see one of those misclassified instances.
The post was labeled as negative by the model, probably because it contains words with negative connotations such as anxiety and negativity, even though the post carries an unequivocally positive message. Also, the post doesn't contain any words that are usually considered highly positive, so that might have confused the model in this case.

In [48]:
filtered_df_realY_0 = df_mismatches[df_mismatches['real_Y'] == 0]
text_y0_conf1 = filtered_df_realY_0[filtered_df_realY_0['confidence'] == 1.0]['text'].tolist()

In [54]:
text_y0_conf1_0 = text_y0_conf1[0]
analyse_text(text_y0_conf1_0)

Predicted class is: NEGATIVE
Probability for NEGATIVE  is 0.97, and probability for POSITIVE is 0.03


In [55]:
text_y0_conf1[0]

'*"When thinking about life, remember this: No amount of guilt can solve the past, and no amount of anxiety can change the future. "* This is a quote I stumbled upon a few years ago that still sticks with me to this very day. At face value it presents the idea that anxiety is fueled by the very nature of what it\'s foundation is based on; **nothing. ** I keep an open mind these days, in fact, one can argue a quite possibly *too* open mind.'

In the example bellow we can see another example where the message was not overall negative, but it contains words such as hate, homeless, overweight, that usually have a negative connotation.

In [62]:
text_y0_conf1_4 = text_y0_conf1[4]
analyse_text(text_y0_conf1_4)

Predicted class is: NEGATIVE
Probability for NEGATIVE  is 1.00, and probability for POSITIVE is 0.00


In [65]:
text_y0_conf1[4]

"So, I've been homeless since about the first, but I was expecting this. What I wasn't expecting was the changes I've made in my life since becoming homeless. Thankfully I have a job, so I do have money. But I hate everywhere that's affordable to eat! I've been a big junk food/fast food fan for years, so I'm a bit overweight, but lately, I just don't seem to care for meat as much, and fast food is just so.. gross right now."

In [71]:
analyse_text("homeless")

Predicted class is: POSITIVE
Probability for NEGATIVE  is 0.00, and probability for POSITIVE is 1.00


In [69]:
analyse_text("overweight")

Predicted class is: NEGATIVE
Probability for NEGATIVE  is 1.00, and probability for POSITIVE is 0.00


In [85]:
filtered_df_realY_1 = df_mismatches[df_mismatches['real_Y'] == 1]
text_y1_conf1 = filtered_df_realY_1[filtered_df_realY_1['confidence'] == 1.000000]['text'].tolist()

Bellow we can see two examples that were classified as stressful, but the raw model classifies one of them as positive and the other one as negative.

The system might have been fooled by some positive statements ('want to be there to support them'), even thought the sentiment is quite negative.

It is impossible to determine what made the additionally trained model choose differently for the second observation, because it is clearly charged with negative emotions.

In [98]:
text_y1_conf1_1 = text_y1_conf1[0]
analyse_text(text_y1_conf1_1)

Predicted class is: POSITIVE
Probability for NEGATIVE  is 0.23, and probability for POSITIVE is 0.77


In [99]:
text_y1_conf1[0]

"I hate this feeling. I hate weddings, but I want to be there to support them... but I can't. <url> &#x200B; edited; For clarity"

In [101]:
analyse_text("I hate weddings, but I want to be there to support them.")

Predicted class is: POSITIVE
Probability for NEGATIVE  is 0.00, and probability for POSITIVE is 1.00


In [102]:
text_y1_conf1_2 = text_y1_conf1[1]
analyse_text(text_y1_conf1_2)

Predicted class is: NEGATIVE
Probability for NEGATIVE  is 1.00, and probability for POSITIVE is 0.00


In [91]:
text_y1_conf1[1]

'Then I came home.      My Mom pointed it out first, I went from being the class clown and the life of the party, to being the quiet guy who stood in the corner of the room. I went from a musician and avid gamer, to having no interest in any of it, and no replacement hobby. The things I had the most passion for in life were gone. It was like someone removed one of my five senses and my soul at the same time.'

**Conclusion**


As anticipated, it is hard to determine what exactly caused the misclassification, but there might be a pattern when it comes to misclassified instances whose true label is 0. It seems that words that carry negative connotation derail the weights in the wrong direction. There are also cases that are very difficult to explain. The analysis was performed using the raw (untrained) model, which is not suitable for stress detection.  

The training of the baseline DL model involved using tokenized posts from around 70% of the dataset, as the training was already very slow. It didn't involve any other fields. Accuracy and other metrics would exhibit much better results, if more parameteres were used, and if the whole dataset was put to use as well.